In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext

import tiktoken

In [6]:
import os
from torchtext.datasets import IWSLT2016
train_iter, valid_iter, test_iter = IWSLT2016()
src_sentence, tgt_sentence = next(iter(train_iter))

HTTPError: 404 Client Error: Not Found for url: https://drive.google.com/uc?id=1l5y6Giag9aRPwGtuZHswh3w5v3qEz8D8
This exception is thrown by __iter__ of GDriveReaderDataPipe(skip_on_error=False, source_datapipe=OnDiskCacheHolderIterDataPipe, timeout=None)

In [5]:
train_iter

(ShardingFilterIterDataPipe,
 ShardingFilterIterDataPipe,
 ShardingFilterIterDataPipe)